In [4]:
from AGiXT import AGiXT, CustomPrompt

agent_name = "gpt4free"
ai = AGiXT(agent_name)

CustomPrompt().add_prompt(
    prompt_name="Write a blog post",
    prompt="Write a blog post for the {industry} instustry. For context, here is a description of my business: {business_description}",
)

response = ai.run(
    task="",
    prompt="Write a blog post",
    industry="software",
    business_description="I am a software company that makes software for software companies.",
)
print(response)


2023-05-27 01:16:25,035 | INFO | KWARGS: {'industry': 'software', 'business_description': 'I am a software company that makes software for software companies.'}
2023-05-27 01:16:25,037 | INFO | Running Chroma using direct local API.
2023-05-27 01:16:25,038 | WARNING | Using embedded DuckDB with persistence: data will be stored in: agents/gpt4free/memories
2023-05-27 01:16:25,042 | INFO | Successfully imported ClickHouse Connect C data optimizations
2023-05-27 01:16:25,043 | INFO | Successfully import ClickHouse Connect C/Numpy optimizations
2023-05-27 01:16:25,045 | INFO | Using orjson library for writing JSON byte strings
2023-05-27 01:16:25,071 | INFO | loaded in 0 embeddings
2023-05-27 01:16:25,072 | INFO | loaded in 1 collections
2023-05-27 01:16:25,454 | INFO | Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-05-27 01:16:25,507 | INFO | Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-05-27 01:16:25,790 | INFO | FORMATTED PROMPT: Write a blog post for the sof

As a software company that specializes in creating software for other software companies, we understand the unique challenges and complexities that arise in this industry. From the constantly evolving technologies to the fast-paced development processes, it’s not always easy to keep up with the demands of the industry.

That’s why we’re dedicated to crafting top-quality software solutions that can help streamline and optimize the development process for our clients. Our software is specifically tailored to meet the needs of software development teams, making their work more efficient, productive, and enjoyable.

Whether you’re a large enterprise software company or a small startup, we know that you’re always looking for ways to stay ahead of the curve. That’s why we put in the time and effort to stay abreast of the latest trends and best practices in software development, so we can create software solutions that are always on the cutting edge.

Our software offerings are designed to he